Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import os
import pandas as pd

path = os.getcwd()
path = path + '\dataset'
df = pd.read_csv(path + '\Fake news classifier train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df.shape

(20800, 5)

In [3]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
df = df.dropna()

In [6]:
df.shape

(18285, 5)

In [7]:
# getting the independent features
X = df.drop('label', axis=1)
# getting the dependent feature
y = df['label']

print(X.shape)
print(y.shape)

(18285, 4)
(18285,)


In [8]:
import tensorflow as tf

In [9]:
print(tf.__version__)

2.11.0


In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
voc_size=5000 

### One hot representation

In [13]:
messages=X.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [14]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [27]:
messages.reset_index(inplace=True)

In [20]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer ##stemming purpose

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lmbmo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Preprocessing and stemming

In [28]:
### Dataset Preprocessing
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    # print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [29]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [30]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [31]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3801, 2787, 4948, 2186, 109, 4397, 1508, 1430, 1447, 2001],
 [331, 924, 635, 3146, 2227, 4015, 4689],
 [1402, 3275, 958, 799],
 [1035, 2857, 2530, 3312, 2108, 2977],
 [4292, 2227, 3901, 4794, 868, 2339, 2227, 1716, 2466, 1467],
 [2312,
  1694,
  2465,
  4540,
  2475,
  2135,
  2905,
  4798,
  2895,
  2395,
  3459,
  1256,
  1313,
  3324,
  4689],
 [2053, 3873, 3589, 2859, 868, 15, 3411, 4233, 4988, 4541, 2709],
 [3271, 3049, 1224, 4850, 74, 3552, 2135, 3046, 4988, 4541, 2709],
 [4225, 1115, 4183, 4280, 545, 3966, 103, 4286, 2135, 4598],
 [3505, 4918, 455, 1488, 424, 3984, 1821, 3483],
 [1840, 1699, 2687, 1723, 1880, 4755, 4845, 3068, 1049, 3162, 3069],
 [3312, 3713, 109, 3966, 2135, 74],
 [3418, 41, 229, 1468, 2197, 420, 4970, 1678, 3330],
 [4405, 3307, 3308, 1534, 752, 3591, 1164, 4988, 4541, 2709],
 [1338, 4274, 2008, 3523, 3753, 4988, 4541, 2709],
 [1225, 3745, 624, 628, 2926, 4563, 3163, 2736, 1501, 3771],
 [1470, 486, 924],
 [3994, 629, 1850, 355, 2135, 4571, 2905, 4689],
 [1369

In [33]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [32]:
onehot_repr[1]

[331, 924, 635, 3146, 2227, 4015, 4689]

### Embedding Representation

In [34]:
sent_length=25
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[3801 2787 4948 ...    0    0    0]
 [ 331  924  635 ...    0    0    0]
 [1402 3275  958 ...    0    0    0]
 ...
 [1204 4531 1800 ...    0    0    0]
 [2466   74 2422 ...    0    0    0]
 [4080 3494 2842 ...    0    0    0]]


### Creating model

In [35]:
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
len(embedded_docs),y.shape

(18285, (18285,))

In [37]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [38]:
X_final.shape,y_final.shape

((18285, 25), (18285,))

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=33)

### Model training

In [40]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 12s 40ms/step - loss: 0.3171 - accuracy: 0.8399 - val_loss: 0.2060 - val_accuracy: 0.9167
Epoch 2/10
200/200 [==============================] - 6s 32ms/step - loss: 0.1382 - accuracy: 0.9480 - val_loss: 0.1996 - val_accuracy: 0.9216
Epoch 3/10
200/200 [==============================] - 6s 31ms/step - loss: 0.0974 - accuracy: 0.9638 - val_loss: 0.2403 - val_accuracy: 0.9213
Epoch 4/10
200/200 [==============================] - 6s 30ms/step - loss: 0.0747 - accuracy: 0.9738 - val_loss: 0.2675 - val_accuracy: 0.9158
Epoch 5/10
200/200 [==============================] - 6s 29ms/step - loss: 0.0576 - accuracy: 0.9802 - val_loss: 0.2894 - val_accuracy: 0.9189
Epoch 6/10
200/200 [==============================] - 6s 29ms/step - loss: 0.0411 - accuracy: 0.9867 - val_loss: 0.3401 - val_accuracy: 0.9182
Epoch 7/10
200/200 [==============================] - 6s 28ms/step - loss: 0.0351 - accuracy: 0.9873 - val_loss: 0.3777 - val_accuracy: 0.908

In [41]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance metrics


In [42]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

172/172 [==============================] - 2s 7ms/step


In [46]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
confusion_matrix(y_test,y_pred)

array([[3156,    0],
       [2330,    0]], dtype=int64)

In [44]:
accuracy_score(y_test,y_pred)

0.5752825373678454

In [47]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.58      1.00      0.73      3156
           1       0.00      0.00      0.00      2330

    accuracy                           0.58      5486
   macro avg       0.29      0.50      0.37      5486
weighted avg       0.33      0.58      0.42      5486



c:\Users\lmbmo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lmbmo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lmbmo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,